# NLP for Business — Reproducible Learning Journey

## Overview
This notebook documents hands-on NLP workflows for analyzing text using:
- **Preprocessing** (cleaning, stopwords, regex)
- **Frequency analysis** (before/after cleaning)
- **Topic modeling (LDA)** (hyperparameter tuning, coherence evaluation)
- **Visualization** (pyLDAvis interactive exploration)

**Dataset:** 20 Newsgroups (a classic NLP corpus with 20 categories)

**Key Learning:** *"Clean text determines meaningful topics."* Most NLP performance gains come from ruthless text cleaning, not fancy algorithms.

---

## Learning Modules Covered
1. **Module 1.2** — Data Preparation & Source Consistency
2. **Module 1.3** — How LDA Topic Modeling Works
3. **Module 1.4** — Stopwords & Frequency Analysis
4. **Module 1.5** — Topic Modeling Experiments
5. **Module 2.1** — Regex Text Cleaning

## Section 1: Import Required Libraries

Import all necessary packages for NLP analysis, topic modeling, and visualization.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import re
import time
import pickle
import collections
from collections import Counter

# NLP and text processing
import tomotopy as tp
import gensim
import gensim.corpora
import gensim.models.coherencemodel
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis

# Display settings
pd.set_option('display.max_colwidth', 200)
sns.set_style('whitegrid')

print("✓ All libraries imported successfully!")
print(f"Tomotopy version: {tp.__version__}")

## Section 2: Load and Explore Dataset

**Module 1.2 Lesson:** *Source consistency matters before modeling*

We'll use the 20 Newsgroups dataset as our corpus. This has 20 different categories of documents - similar to how companies span different industries.

In [ ]:
# Load 20 Newsgroups dataset (removing headers, footers, quotes for cleaner text)
print("Loading 20 Newsgroups dataset...")
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Create DataFrame
df = pd.DataFrame({
    'text': newsgroups.data,
    'category': [newsgroups.target_names[i] for i in newsgroups.target]
})

print(f"✓ Loaded {len(df)} documents")
print(f"✓ {len(newsgroups.target_names)} categories\n")
print("Categories:", newsgroups.target_names)

In [ ]:
# Basic exploration
print("=== DATASET OVERVIEW ===")
print(df.info())
print("\n=== SAMPLE DOCUMENTS ===")
print(df.head(3))

# Add word count
df['nWords'] = df['text'].str.split().apply(len)

print("\n=== DOCUMENT LENGTH STATISTICS ===")
print(df['nWords'].describe())

# Visualize length distribution
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.hist(df['nWords'], bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.title('Document Length Distribution')
plt.axvline(df['nWords'].median(), color='red', linestyle='--', label=f'Median: {df["nWords"].median():.0f}')
plt.legend()

plt.subplot(1, 2, 2)
category_counts = df['category'].value_counts()
plt.barh(category_counts.head(10).index, category_counts.head(10).values)
plt.xlabel('Number of Documents')
plt.title('Top 10 Categories by Document Count')
plt.tight_layout()
plt.show()

## Section 3: Raw Word Frequency Analysis

**Module 1.4 Lesson:** *Raw frequencies reflect language, not meaning*

Before any cleaning, let's see what words dominate the corpus. We expect to see mostly grammar words like "the", "and", "of", etc.

In [ ]:
# Tokenize all documents (raw, no preprocessing)
all_words_raw = []
for text in df['text']:
    words = str(text).lower().split()
    all_words_raw.extend(words)

# Count frequencies
word_freq_raw = Counter(all_words_raw)
top_20_raw = word_freq_raw.most_common(20)

print("=== TOP 20 MOST COMMON WORDS (RAW TEXT) ===")
for word, count in top_20_raw:
    print(f"{word:20s} {count:,}")

# Visualize
plt.figure(figsize=(12, 6))
words, counts = zip(*top_20_raw)
plt.barh(range(len(words)), counts, color='steelblue')
plt.yticks(range(len(words)), words)
plt.xlabel('Frequency')
plt.title('Top 20 Words (Raw Text) - Dominated by Grammar Words', fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

print(f"\n💡 Notice: The top words are meaningless grammar words!")
print("   These words tell us nothing about document content.")

## Section 4: Text Preprocessing Pipeline

**Key Insight:** *Preprocessing dominates results*

We'll implement a multi-stage cleaning pipeline:
1. Lowercase conversion
2. Remove URLs and email addresses
3. Remove numbers
4. Remove special characters
5. Remove stopwords (generic + custom)

In [ ]:
def clean_text(text, remove_stopwords=False, custom_stopwords=None):
    """
    Clean and preprocess text
    
    Args:
        text: Raw text string
        remove_stopwords: Whether to remove stopwords
        custom_stopwords: Additional stopwords to remove
    
    Returns:
        Cleaned text string
    """
    # Convert to lowercase
    text = str(text).lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove numbers (Module 2.1 lesson)
    text = re.sub(r'\d+', '', text)
    
    # Remove special characters and extra whitespace
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove stopwords if requested
    if remove_stopwords:
        words = text.split()
        
        # Start with sklearn's ENGLISH_STOP_WORDS
        stopwords = set(ENGLISH_STOP_WORDS)
        
        # Add custom stopwords if provided
        if custom_stopwords:
            stopwords.update(custom_stopwords)
        
        # Filter out stopwords and very short words
        words = [w for w in words if w not in stopwords and len(w) > 2]
        text = ' '.join(words)
    
    return text

# Test the function on a sample document
sample_idx = 0
sample_text = df.loc[sample_idx, 'text']

print("=== BEFORE CLEANING ===")
print(sample_text[:500])
print("\n=== AFTER CLEANING (no stopword removal) ===")
print(clean_text(sample_text, remove_stopwords=False)[:500])
print("\n=== AFTER CLEANING (with stopword removal) ===")
print(clean_text(sample_text, remove_stopwords=True)[:500])

## Section 5: Stopword Removal & Custom Filtering

**Module 1.4 Lesson:** *Removing stopwords shifts signals from grammar to content*

Generic stopwords aren't enough. We need **domain-specific** stopwords too.

In [ ]:
# Apply cleaning with stopword removal
df['cleanText'] = df['text'].apply(lambda x: clean_text(x, remove_stopwords=True))

# Add word count after cleaning
df['nWords_clean'] = df['cleanText'].str.split().apply(len)

# Filter out very short documents
df = df[df['nWords_clean'] > 10].reset_index(drop=True)

print(f"After filtering: {len(df)} documents remaining")
print(f"Average words per document (cleaned): {df['nWords_clean'].mean():.1f}")

# Recompute word frequencies on cleaned text
all_words_clean = []
for text in df['cleanText']:
    words = text.split()
    all_words_clean.extend(words)

word_freq_clean = Counter(all_words_clean)
top_20_clean = word_freq_clean.most_common(20)

print("\n=== TOP 20 MOST COMMON WORDS (CLEANED TEXT) ===")
for word, count in top_20_clean:
    print(f"{word:20s} {count:,}")

In [ ]:
# Compare raw vs cleaned frequencies side-by-side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Raw frequencies
words_raw, counts_raw = zip(*top_20_raw)
ax1.barh(range(len(words_raw)), counts_raw, color='lightcoral')
ax1.set_yticks(range(len(words_raw)))
ax1.set_yticklabels(words_raw)
ax1.set_xlabel('Frequency')
ax1.set_title('Top 20 Words - RAW TEXT\n(Grammar words dominate)', fontweight='bold', fontsize=12)
ax1.invert_yaxis()

# Cleaned frequencies
words_clean, counts_clean = zip(*top_20_clean)
ax2.barh(range(len(words_clean)), counts_clean, color='lightgreen')
ax2.set_yticks(range(len(words_clean)))
ax2.set_yticklabels(words_clean)
ax2.set_xlabel('Frequency')
ax2.set_title('Top 20 Words - CLEANED TEXT\n(Content words emerge)', fontweight='bold', fontsize=12)
ax2.invert_yaxis()

plt.tight_layout()
plt.show()

print("\n💡 KEY OBSERVATION:")
print("   After cleaning, meaningful content words surface!")
print("   Words like 'people', 'think', 'know', 'just' are more informative.")

## Section 6: Helper Functions for LDA

**Module 1.3 Lesson:** *LDA discovers latent themes via probabilistic word co-occurrence*

LDA treats:
- Each **document** as a mix of topics
- Each **topic** as a mix of words

We need functions to:
1. Build LDA models
2. Calculate coherence (how interpretable are the topics?)

In [ ]:
def getCoherence(model, ncore=4):
    """
    Calculate coherence score for a tomotopy LDA model
    
    Higher coherence = more interpretable topics
    """
    topics = []
    for k in range(model.k):
        word_probs = model.get_topic_words(k)
        topics.append([word for word, prob in word_probs])
        
    texts = []
    corpus = []
    for doc in model.docs:
        words = [model.vocabs[token_id] for token_id in doc.words]
        texts.append(words)
        freqs = list(collections.Counter(doc.words).items())
        corpus.append(freqs)
    
    id2word = dict(enumerate(model.vocabs))
    dictionary = gensim.corpora.dictionary.Dictionary.from_corpus(
        corpus, id2word
    )
    
    cm = gensim.models.coherencemodel.CoherenceModel(
        topics=topics,
        texts=texts,
        corpus=corpus,
        dictionary=dictionary,
        processes=ncore
    )
    
    return cm.get_coherence()


def buildModel(df, hP, txt_col, k, alpha, eta):
    """
    Build and train an LDA model using tomotopy
    
    Args:
        df: DataFrame with documents
        hP: Hyperparameters dict
        txt_col: Column name with text
        k: Number of topics
        alpha: Document-topic concentration (higher = more topics per doc)
        eta: Topic-word concentration (higher = more words per topic)
    
    Returns:
        Trained LDA model
    """
    model = tp.LDAModel(
        alpha=alpha,      
        eta=eta,   
        k=k
    )

    df_split = [doc.split() for doc in df[txt_col].astype(str)]
    for doc in df_split: 
        model.add_doc(doc)

    model.train(iter=hP['train_iter'])

    return model

print("✓ Helper functions defined")

## Section 7: Hyperparameter Tuning - Number of Topics (k)

**Goal:** Find the optimal number of topics

**Metrics:**
- **Perplexity:** Lower is better (how well model predicts held-out data)
- **Coherence:** Higher is better (how interpretable are topics)

We'll test k = [10, 15, 20, 25, 30]

In [ ]:
# Hyperparameters
hP = {
    'train_iter': 500,  # Number of training iterations
    'n_cores': 4,       # Parallel processing
}

# Test different numbers of topics
coherence_k = []
perplexity_k = []
ks = [10, 15, 20, 25, 30]
alphas = [0.1]
etas = [0.01]

print("=== TUNING NUMBER OF TOPICS (k) ===\n")
for k in ks:
    for alpha in alphas:
        for eta in etas:
            print(f"Training: k={k}, alpha={alpha}, eta={eta}...", end=' ')
            start_time = time.time()
            
            ldaModel = buildModel(df, hP, 'cleanText', k, alpha, eta)
            p = ldaModel.perplexity
            c = getCoherence(ldaModel, 4)
            
            coherence_k.append([k, alpha, eta, c])
            perplexity_k.append([k, alpha, eta, p])
            
            elapsed = time.time() - start_time
            print(f"Done in {elapsed:.1f}s | Perplexity: {p:.4f}, Coherence: {c:.4f}")

# Find best k
best_k_idx = np.argmax([x[3] for x in coherence_k])
best_k = coherence_k[best_k_idx][0]
print(f"\n✓ Best k (by coherence): {best_k}")

In [ ]:
# Plot results for k tuning
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

X = [x[0] for x in coherence_k]
Y = [y[3] for y in coherence_k]
ax1.plot(X, Y, marker='o', linewidth=2, markersize=8, color='green')
ax1.set_title('Topic Coherence by Number of Topics', fontsize=12, fontweight='bold')
ax1.set_xlabel('Number of Topics (k)', fontsize=11)
ax1.set_ylabel('Coherence Score (higher = better)', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.axvline(best_k, color='red', linestyle='--', alpha=0.7, label=f'Best k={best_k}')
ax1.legend()

X = [x[0] for x in perplexity_k]
Y = [y[3] for y in perplexity_k]
ax2.plot(X, Y, marker='o', linewidth=2, markersize=8, color='orange')
ax2.set_title('Model Perplexity by Number of Topics', fontsize=12, fontweight='bold')
ax2.set_xlabel('Number of Topics (k)', fontsize=11)
ax2.set_ylabel('Perplexity (lower = better)', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETATION:")
print("   - Coherence peaks suggest optimal topic interpretability")
print("   - Perplexity generally decreases with more topics")
print("   - Balance: Choose k with high coherence, reasonable perplexity")

## Section 8: Hyperparameter Tuning - Alpha

**Alpha:** Document-topic distribution parameter
- **Lower alpha** → Documents focus on fewer topics
- **Higher alpha** → Documents spread across many topics

We'll test alpha = [0.01, 0.1, 0.5, 1.0, 2.0] with the best k

In [ ]:
coherence_alphas = []
perplexity_alphas = []
ks = [best_k]
alphas = [0.01, 0.1, 0.5, 1.0, 2.0]
etas = [0.01]

print("=== TUNING ALPHA ===\n")
for k in ks:
    for alpha in alphas:
        for eta in etas:
            print(f"Training: k={k}, alpha={alpha}, eta={eta}...", end=' ')
            start_time = time.time()
            
            ldaModel = buildModel(df, hP, 'cleanText', k, alpha, eta)
            p = ldaModel.perplexity
            c = getCoherence(ldaModel, 4)
            
            coherence_alphas.append([k, alpha, eta, c])
            perplexity_alphas.append([k, alpha, eta, p])
            
            elapsed = time.time() - start_time
            print(f"Done in {elapsed:.1f}s | Perplexity: {p:.4f}, Coherence: {c:.4f}")

# Find best alpha
best_alpha_idx = np.argmax([x[3] for x in coherence_alphas])
best_alpha = coherence_alphas[best_alpha_idx][1]
print(f"\n✓ Best alpha (by coherence): {best_alpha}")

In [ ]:
# Plot alpha tuning results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

X = [x[1] for x in coherence_alphas]
Y = [y[3] for y in coherence_alphas]
ax1.plot(X, Y, marker='o', linewidth=2, markersize=8, color='green')
ax1.set_title('Topic Coherence by Alpha', fontsize=12, fontweight='bold')
ax1.set_xlabel('Alpha (document-topic concentration)', fontsize=11)
ax1.set_ylabel('Coherence Score', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.axvline(best_alpha, color='red', linestyle='--', alpha=0.7, label=f'Best α={best_alpha}')
ax1.legend()

X = [x[1] for x in perplexity_alphas]
Y = [y[3] for y in perplexity_alphas]
ax2.plot(X, Y, marker='o', linewidth=2, markersize=8, color='orange')
ax2.set_title('Model Perplexity by Alpha', fontsize=12, fontweight='bold')
ax2.set_xlabel('Alpha (document-topic concentration)', fontsize=11)
ax2.set_ylabel('Perplexity', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Section 9: Hyperparameter Tuning - Eta

**Eta:** Topic-word distribution parameter
- **Lower eta** → Topics focus on fewer words (more specific)
- **Higher eta** → Topics spread across many words (more general)

We'll test eta = [0.001, 0.01, 0.05, 0.1, 0.2] with best k and alpha

In [ ]:
coherence_etas = []
perplexity_etas = []
ks = [best_k]
alphas = [best_alpha]
etas = [0.001, 0.01, 0.05, 0.1, 0.2]

print("=== TUNING ETA ===\n")
for k in ks:
    for alpha in alphas:
        for eta in etas:
            print(f"Training: k={k}, alpha={alpha}, eta={eta}...", end=' ')
            start_time = time.time()
            
            ldaModel = buildModel(df, hP, 'cleanText', k, alpha, eta)
            p = ldaModel.perplexity
            c = getCoherence(ldaModel, 4)
            
            coherence_etas.append([k, alpha, eta, c])
            perplexity_etas.append([k, alpha, eta, p])
            
            elapsed = time.time() - start_time
            print(f"Done in {elapsed:.1f}s | Perplexity: {p:.4f}, Coherence: {c:.4f}")

# Find best eta
best_eta_idx = np.argmax([x[3] for x in coherence_etas])
best_eta = coherence_etas[best_eta_idx][2]
print(f"\n✓ Best eta (by coherence): {best_eta}")

In [ ]:
# Plot eta tuning results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

X = [x[2] for x in coherence_etas]
Y = [y[3] for y in coherence_etas]
ax1.plot(X, Y, marker='o', linewidth=2, markersize=8, color='green')
ax1.set_title('Topic Coherence by Eta', fontsize=12, fontweight='bold')
ax1.set_xlabel('Eta (topic-word concentration)', fontsize=11)
ax1.set_ylabel('Coherence Score', fontsize=11)
ax1.grid(True, alpha=0.3)
ax1.axvline(best_eta, color='red', linestyle='--', alpha=0.7, label=f'Best η={best_eta}')
ax1.legend()

X = [x[2] for x in perplexity_etas]
Y = [y[3] for y in perplexity_etas]
ax2.plot(X, Y, marker='o', linewidth=2, markersize=8, color='orange')
ax2.set_title('Model Perplexity by Eta', fontsize=12, fontweight='bold')
ax2.set_xlabel('Eta (topic-word concentration)', fontsize=11)
ax2.set_ylabel('Perplexity', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n✅ OPTIMAL HYPERPARAMETERS FOUND:")
print(f"   k (topics): {best_k}")
print(f"   alpha: {best_alpha}")
print(f"   eta: {best_eta}")

## Section 10: Build Final Model with Optimal Parameters

Now we'll train a final model with more iterations using our optimized hyperparameters.

In [ ]:
print("=== BUILDING FINAL MODEL ===")
print(f"Hyperparameters: k={best_k}, alpha={best_alpha}, eta={best_eta}")
print("Training with 1000 iterations...\n")

# Build final model with more training iterations
hP_final = {
    'train_iter': 1000,
    'n_cores': 4,
    'eta': best_eta,
    'alpha': best_alpha,
    'k': best_k
}

def buildFinalModel(df, hP, txt_col):
    """Build final model using parameters from hP dict"""
    model = tp.LDAModel(
        alpha=hP['alpha'],      
        eta=hP['eta'],   
        k=hP['k']
    )

    df_split = [doc.split() for doc in df[txt_col].astype(str)]
    for doc in df_split: 
        model.add_doc(doc)

    model.train(iter=hP['train_iter'])

    return model

start_time = time.time()
final_model = buildFinalModel(df, hP_final, 'cleanText')
elapsed = time.time() - start_time

print(f"✓ Training complete in {elapsed:.1f}s")
print(f"\n### MODEL EVALUATION ###")
print(f"Perplexity: {final_model.perplexity:.4f}")
print(f"Coherence: {getCoherence(final_model, 4):.4f}")
print(f"Total vocabulary size: {len(final_model.vocabs)}")
print(f"Number of documents: {len(final_model.docs)}")

## Section 11: Explore Topics

**Module 1.5 Lesson:** *Coherent topics emerge from strong co-occurrence patterns*

Let's examine what topics our model discovered.

In [ ]:
print("### TOP WORDS PER TOPIC ###\n")
for k in range(final_model.k):
    words = final_model.get_topic_words(k, top_n=10)
    word_list = ', '.join([word for word, prob in words])
    print(f'Topic #{k}: {word_list}')

In [ ]:
# Save the model and data
print("\n=== SAVING MODEL ===")
final_model.save("newsgroups_lda_final.mdl")
pickle.dump(df, open("newsgroups_df.p", "wb"))
print("✓ Model saved as 'newsgroups_lda_final.mdl'")
print("✓ DataFrame saved as 'newsgroups_df.p'")

## Section 12: Interactive Visualization with pyLDAvis

**pyLDAvis** creates an interactive HTML visualization showing:
- Topic clusters in 2D space
- Topic prevalence (size)
- Top terms per topic
- Term frequency distributions

This is the most intuitive way to explore and interpret LDA results!

In [ ]:
print("=== CREATING PYLDAVIS VISUALIZATION ===")

# Prepare data for pyLDAvis
topic_term_dists = np.stack([final_model.get_topic_word_dist(k) for k in range(final_model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in final_model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in final_model.docs])
vocab = list(final_model.used_vocabs)
term_frequency = final_model.used_vocab_freq

print("Preparing interactive visualization...")
prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    start_index=0,  # tomotopy uses 0-based indexing
    sort_topics=False  # Keep topic IDs consistent
)

# Save as HTML
pyLDAvis.save_html(prepared_data, 'newsgroups_ldavis.html')
print("✓ Interactive visualization saved as 'newsgroups_ldavis.html'")
print("\n💡 Open 'newsgroups_ldavis.html' in your browser to explore topics interactively!")

# Display in notebook (if running in Jupyter)
# pyLDAvis.display(prepared_data)

## Section 13: LDA Validation and Analysis

Beyond visual inspection, we need **quantitative validation** of our topics. This section implements systematic checks to validate topic quality and interpretability.

### What We'll Check:
1. **Term-Topic Assignment** - Which topic maximizes each term?
2. **Topic Separation** - Are topics distinct or overlapping?
3. **Term Weights Analysis** - Compare related terms across topics
4. **Model Metrics** - Final perplexity and coherence scores

### 13.1: Find Which Topic Maximizes a Term

For any word, we want to know: **Which topic has the highest P(word | topic)?**

This helps identify topic content without relying on visual inspection.

In [ ]:
def find_term_topic(model, term, top_n=500):
    """
    Find the topic where a term has the highest probability P(word | topic)
    
    Args:
        model: Trained tomotopy LDA model
        term: Word to search for
        top_n: Number of top words to check per topic
    
    Returns:
        (best_topic_id, best_probability)
    """
    best_topic = None
    best_prob = 0.0
    
    for k in range(model.k):
        words = dict(model.get_topic_words(k, top_n=top_n))
        if term in words and words[term] > best_prob:
            best_prob = words[term]
            best_topic = k
    
    return best_topic, best_prob

# Test with some example terms from the dataset
test_terms = ["computer", "game", "space", "religion", "medical", "car", "politics"]

print("=== TERM-TOPIC ASSIGNMENTS ===\n")
for term in test_terms:
    topic_id, prob = find_term_topic(final_model, term)
    if topic_id is not None:
        print(f"{term:15s} -> Topic {topic_id:2d} (P={prob:.5f})")
        # Show top 5 words for that topic
        top_words = ', '.join([w for w, p in final_model.get_topic_words(topic_id, top_n=5)])
        print(f"                Top words: {top_words}\n")
    else:
        print(f"{term:15s} -> Not found in top {500} words\n")

### 13.2: Compare Related Terms Across Topics

For semantically related terms, we want to see which topics emphasize which words. This reveals topic specialization.

In [ ]:
def term_weights_by_topic(model, terms, topn=500):
    """
    Get P(word | topic) for multiple terms across all topics
    
    Args:
        model: Trained tomotopy LDA model
        terms: List of terms to analyze
        topn: Number of top words to check per topic
    
    Returns:
        Dict mapping topic_id -> {term: probability}
    """
    out = {}
    for k in range(model.k):
        w = dict(model.get_topic_words(k, top_n=topn))
        hits = {t: w.get(t, 0.0) for t in terms if t in w}
        if hits:
            out[k] = hits
    return out

# Example: Analyze technology-related terms
tech_terms = ["computer", "software", "hardware", "programming", "algorithm", "data"]

print("=== TERM WEIGHTS ACROSS TOPICS (Technology Terms) ===\n")
tw = term_weights_by_topic(final_model, tech_terms)

for term in tech_terms:
    # Rank topics by this term's probability
    ranked = sorted(
        [(k, tw[k].get(term, 0.0)) for k in tw if term in tw[k]],
        key=lambda x: x[1],
        reverse=True
    )
    
    if ranked:
        print(f"\n{term}:")
        for k, val in ranked[:3]:  # Top 3 topics
            print(f"  Topic {k:2d}: {val:.5f}")
            
# Show detailed view of the top topic for "computer"
print("\n" + "="*60)
print("DETAILED VIEW: Topic containing 'computer'")
print("="*60)
comp_topic, _ = find_term_topic(final_model, "computer")
if comp_topic is not None:
    print(f"\nTopic {comp_topic} - Top 15 words:")
    for word, prob in final_model.get_topic_words(comp_topic, top_n=15):
        print(f"  {word:20s} {prob:.5f}")

### 13.3: Topic Separation Metric

**Quantify topic overlap** using cosine similarity between topic-word distributions.

- **Lower average similarity** = more distinct topics
- **Higher average similarity** = overlapping/redundant topics

**Rule of thumb:**
- < 0.3: Topics are highly distinct
- 0.3-0.5: Moderate separation
- > 0.5: Topics are overlapping

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Get topic-word distributions for all topics
topic_dists = np.stack([
    final_model.get_topic_word_dist(k)
    for k in range(final_model.k)
])

# Calculate cosine similarity between all topic pairs
sim_matrix = cosine_similarity(topic_dists)

# Calculate average off-diagonal similarity
k = final_model.k
avg_sim = (sim_matrix.sum() - k) / (k * (k - 1))

print("=== TOPIC SEPARATION ANALYSIS ===\n")
print(f"Number of topics: {k}")
print(f"Average topic cosine similarity: {avg_sim:.4f}")

if avg_sim < 0.3:
    print("✓ EXCELLENT: Topics are highly distinct")
elif avg_sim < 0.5:
    print("✓ GOOD: Topics have moderate separation")
else:
    print("⚠ WARNING: Topics may be overlapping")

# Find most similar topic pairs
print("\n=== MOST SIMILAR TOPIC PAIRS ===")
similar_pairs = []
for i in range(k):
    for j in range(i+1, k):
        similar_pairs.append((i, j, sim_matrix[i, j]))

similar_pairs.sort(key=lambda x: x[2], reverse=True)

print("\nTop 5 most similar topic pairs:")
for i, j, sim in similar_pairs[:5]:
    print(f"\nTopics {i} & {j} (similarity: {sim:.4f})")
    print(f"  Topic {i}: {', '.join([w for w, p in final_model.get_topic_words(i, top_n=5)])}")
    print(f"  Topic {j}: {', '.join([w for w, p in final_model.get_topic_words(j, top_n=5)])}")

# Visualize similarity matrix
plt.figure(figsize=(10, 8))
plt.imshow(sim_matrix, cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Cosine Similarity')
plt.title('Topic Similarity Matrix\n(Darker = More Similar)', fontweight='bold', fontsize=14)
plt.xlabel('Topic ID', fontsize=12)
plt.ylabel('Topic ID', fontsize=12)
plt.tight_layout()
plt.show()

### 13.4: Final Model Quality Metrics

Summary of all key metrics for the final model.

In [ ]:
print("="*70)
print("FINAL MODEL QUALITY REPORT")
print("="*70)

print("\n📊 MODEL CONFIGURATION")
print(f"  Number of topics (k):        {best_k}")
print(f"  Alpha (doc-topic):           {best_alpha}")
print(f"  Eta (topic-word):            {best_eta}")
print(f"  Training iterations:         {hP_final['train_iter']}")

print("\n📈 PERFORMANCE METRICS")
final_perplexity = final_model.perplexity
final_coherence = getCoherence(final_model, 4)
print(f"  Perplexity (lower=better):   {final_perplexity:.4f}")
print(f"  Coherence (higher=better):   {final_coherence:.4f}")
print(f"  Topic separation (avg sim):  {avg_sim:.4f}")

print("\n📚 CORPUS STATISTICS")
print(f"  Documents processed:         {len(final_model.docs):,}")
print(f"  Vocabulary size:             {len(final_model.vocabs):,}")
print(f"  Used vocabulary:             {len(final_model.used_vocabs):,}")

print("\n✅ QUALITY ASSESSMENT")
if final_coherence > 0.4:
    print("  ✓ Coherence: EXCELLENT")
elif final_coherence > 0.3:
    print("  ✓ Coherence: GOOD")
else:
    print("  ⚠ Coherence: NEEDS IMPROVEMENT")

if avg_sim < 0.3:
    print("  ✓ Topic Separation: EXCELLENT")
elif avg_sim < 0.5:
    print("  ✓ Topic Separation: GOOD")
else:
    print("  ⚠ Topic Separation: NEEDS IMPROVEMENT")

print("\n" + "="*70)
print("💡 INTERPRETATION TIPS:")
print("="*70)
print("  • Use pyLDAvis HTML to explore topics interactively")
print("  • Check term-topic assignments to understand topic content")
print("  • Review topic separation matrix for redundant topics")
print("  • Consider retraining with different k if topics overlap heavily")
print("="*70)

### 13.5: Common Pitfalls to Avoid

⚠️ **Important Reminders:**

1. **Don't identify topics by pyLDAvis bubble position alone**
   - Always click the bubble and read the term bars on the right
   - Or use the `find_term_topic()` function to identify topics programmatically

2. **Ensure your final model uses the parameters you claim**
   - Check that k, alpha, eta match what you report
   - Verify training iterations were actually applied

3. **Burn-in doesn't apply automatically**
   - If you set `burn_in` in a config dict, you must apply it:
   ```python
   model.burn_in = hP["burn_in"]
   ```

4. **High coherence alone isn't enough**
   - Also check topic separation
   - Review actual term-topic assignments
   - Validate with domain knowledge

5. **Document your evidence**
   - Save term-topic maxima for key terms
   - Record topic separation metrics
   - Export and version control your pyLDAvis HTML

## Section 14: Key Takeaways & Lessons Learned

### Core Lessons from This Analysis

1. **Preprocessing dominates results**
   - Most performance gains came from text cleaning, not modeling
   - Stopword removal is critical
   - Domain-specific filtering matters

2. **Generic language is the enemy**
   - Grammar words hide meaningful content
   - Custom stopwords are essential
   - Regex cleaning removes noise (numbers, dates)

3. **Hyperparameter tuning is worth it**
   - Different k values produce very different topics
   - Alpha and eta fine-tune topic granularity
   - Coherence helps find optimal parameters

4. **Context matters**
   - More text per document → better topics
   - Short descriptions struggle to show clear themes
   - Combining sources improves interpretability

5. **Domain knowledge beats generic NLP**
   - Understanding your data helps filter noise
   - Custom preprocessing pipelines are necessary
   - One-size-fits-all solutions don't work

6. **Validation is essential**
   - Don't rely on visualizations alone
   - Use quantitative metrics (coherence, separation)
   - Check term-topic assignments programmatically
   - Document your evidence

---

### The NLP Pipeline for Text Analysis

```
Raw Text
  ↓
Source Standardization
  ↓
Lowercasing
  ↓
Remove URLs, emails, special chars
  ↓
Remove numbers (if appropriate)
  ↓
Stopword Removal (generic + domain-specific)
  ↓
Frequency Analysis
  ↓
Topic Modeling (LDA)
  ↓
Hyperparameter Tuning
  ↓
Evaluation & Interpretation
  ↓
Visualization (pyLDAvis)
  ↓
Validation (term-topic, separation metrics)
```

---

### Final Insight

> **"Topic modeling on text is less about fancy algorithms and more about ruthless text cleaning. The model only reflects whatever language survives preprocessing."**

> **"Visual inspection is not enough. Always validate with quantitative metrics and term-topic assignments."**

**Clean text + rigorous validation = meaningful topics.**